In [13]:
!cd Soil-link/ && git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 2.59 MiB | 2.13 MiB/s, done.
From https://github.com/AnasAlmana/Soil-link
   e9c336c..e046239  main       -> origin/main
Updating e9c336c..e046239
Fast-forward
 composter_moisture_prediction_dataset.csv | 100000 +++++++++++++--------------
 1 file changed, 50000 insertions(+), 50000 deletions(-)


In [14]:
import pandas as pd

df = pd.read_csv("/content/Soil-link/composter_moisture_prediction_dataset.csv")
df.head()

,timestamp,event,temperature_C,CO2_ppm,moisture_percent,mass_loss_percent,pH,forecast_temp_C,forecast_humidity_percent,forecast_condition,humidity_sensor_percent,gps_latitude,gps_longitude,remaining_hours
0,2025-01-01 00:00:00,mixing,50.55,1207.3,65.726870,0.96,7.50,28.53,36.3,rainy,58.6,59.122015,-169.234622,103.1
1,2025-01-01 00:05:00,mixing,55.53,1487.4,60.652572,0.27,6.86,24.41,58.9,foggy,65.2,44.658528,-117.128323,97.1
2,2025-01-01 00:10:00,mixing,41.49,1373.5,60.774765,0.26,6.49,34.03,70.8,hazy,63.6,22.842842,118.471462,95.6
3,2025-01-01 00:15:00,idle,42.47,1657.6,58.845599,0.27,6.82,22.41,45.2,clear,63.1,55.515698,-56.073778,90.3
4,2025-01-01 00:20:00,idle,56.58,849.5,61.746047,0.96,7.18,31.93,65.9,cloudy,61.9,-29.460187,80.000731,98.8


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timestamp                  50000 non-null  object 
 1   event                      50000 non-null  object 
 2   temperature_C              50000 non-null  float64
 3   CO2_ppm                    50000 non-null  float64
 4   moisture_percent           50000 non-null  float64
 5   mass_loss_percent          50000 non-null  float64
 6   pH                         50000 non-null  float64
 7   forecast_temp_C            50000 non-null  float64
 8   forecast_humidity_percent  50000 non-null  float64
 9   forecast_condition         50000 non-null  object 
 10  humidity_sensor_percent    50000 non-null  float64
 11  gps_latitude               50000 non-null  float64
 12  gps_longitude              50000 non-null  float64
 13  remaining_hours            50000 non-null  flo

In [16]:
df['event'].unique()

array(['mixing', 'idle', 'water_spritzing', 'venting'], dtype=object)

In [17]:
df['forecast_condition'].unique()

array(['rainy', 'foggy', 'hazy', 'clear', 'cloudy', 'snowy', 'stormy',
       'partly cloudy', 'sunny', 'windy'], dtype=object)

In [20]:
df.describe()

,temperature_C,CO2_ppm,moisture_percent,mass_loss_percent,pH,forecast_temp_C,forecast_humidity_percent,humidity_sensor_percent,gps_latitude,gps_longitude,remaining_hours
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,44.944816,1500.421428,37.047508,15.479724,6.999414,34.790498,55.333734,37.153636,-0.129591,-0.124663,60.594896
std,7.966315,496.111269,15.122936,8.655253,0.501707,9.360993,19.058466,15.737113,51.997814,104.022426,22.864414
min,20.000000,300.000000,5.000000,0.040000,5.500000,5.000000,20.000000,10.000000,-89.997065,-179.998342,8.400000
25%,39.600000,1162.950000,24.559237,7.990000,6.660000,28.360000,41.700000,24.400000,-45.074706,-90.721204,41.800000
50%,44.920000,1499.000000,37.019142,15.480000,7.000000,35.130000,55.000000,37.000000,-0.328651,-0.209067,60.600000
75%,50.270000,1834.825000,49.572612,22.990000,7.340000,41.730000,68.600000,49.600000,45.011509,89.702341,79.400000
max,70.000000,3000.000000,70.000000,30.000000,8.500000,50.000000,100.000000,85.700000,89.999823,179.998535,120.000000


## Training:

### Data preparation:

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.set_option('display.max_columns', None)
# --- Load dataset ---
df = pd.read_csv("/content/Soil-link/composter_moisture_prediction_dataset.csv")

# --- Feature Engineering on timestamp ---
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek

# Encode time as cyclical features
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

# Drop unused
df.drop(columns=['timestamp', 'hour', 'dayofweek'], inplace=True)

# One-hot encode categoricals
df = pd.get_dummies(df, columns=['event', 'forecast_condition'])

df.head()


,temperature_C,CO2_ppm,moisture_percent,mass_loss_percent,pH,forecast_temp_C,forecast_humidity_percent,humidity_sensor_percent,gps_latitude,gps_longitude,remaining_hours,hour_sin,hour_cos,dayofweek_sin,dayofweek_cos,event_idle,event_mixing,event_venting,event_water_spritzing,forecast_condition_clear,forecast_condition_cloudy,forecast_condition_foggy,forecast_condition_hazy,forecast_condition_partly cloudy,forecast_condition_rainy,forecast_condition_snowy,forecast_condition_stormy,forecast_condition_sunny,forecast_condition_windy
0,50.55,1207.3,65.726870,0.96,7.50,28.53,36.3,58.6,59.122015,-169.234622,103.1,0.0,1.0,0.974928,-0.222521,False,True,False,False,False,False,False,False,False,True,False,False,False,False
1,55.53,1487.4,60.652572,0.27,6.86,24.41,58.9,65.2,44.658528,-117.128323,97.1,0.0,1.0,0.974928,-0.222521,False,True,False,False,False,False,True,False,False,False,False,False,False,False
2,41.49,1373.5,60.774765,0.26,6.49,34.03,70.8,63.6,22.842842,118.471462,95.6,0.0,1.0,0.974928,-0.222521,False,True,False,False,False,False,False,True,False,False,False,False,False,False
3,42.47,1657.6,58.845599,0.27,6.82,22.41,45.2,63.1,55.515698,-56.073778,90.3,0.0,1.0,0.974928,-0.222521,True,False,False,False,True,False,False,False,False,False,False,False,False,False
4,56.58,849.5,61.746047,0.96,7.18,31.93,65.9,61.9,-29.460187,80.000731,98.8,0.0,1.0,0.974928,-0.222521,True,False,False,False,False,True,False,False,False,False,False,False,False,False


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   temperature_C                     50000 non-null  float64
 1   CO2_ppm                           50000 non-null  float64
 2   moisture_percent                  50000 non-null  float64
 3   mass_loss_percent                 50000 non-null  float64
 4   pH                                50000 non-null  float64
 5   forecast_temp_C                   50000 non-null  float64
 6   forecast_humidity_percent         50000 non-null  float64
 7   humidity_sensor_percent           50000 non-null  float64
 8   gps_latitude                      50000 non-null  float64
 9   gps_longitude                     50000 non-null  float64
 10  remaining_hours                   50000 non-null  float64
 11  hour_sin                          50000 non-null  float64
 12  hour

### Model Training:

In [43]:

# Split features and target
X = df.drop(columns=['remaining_hours'])
y = df['remaining_hours']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

(mae, rmse, r2)

(2.4163405000000004, np.float64(3.0343336921143003), 0.9821457944393627)

In [45]:
import joblib
joblib.dump(model, "remaining_hours_predictor.pkl")

['remaining_hours_predictor.pkl']

In [46]:
# Inference
import joblib
model = joblib.load("remaining_hours_predictor.pkl")
def preprocess_input_with_time_gps(input_row):
    """
    input_row: dict with raw inputs including timestamp and gps
    returns: preprocessed DataFrame ready for prediction
    """

    # Timestamp parsing
    ts = pd.to_datetime(input_row['timestamp'])
    input_row['hour_sin'] = np.sin(2 * np.pi * ts.hour / 24)
    input_row['hour_cos'] = np.cos(2 * np.pi * ts.hour / 24)
    input_row['dayofweek_sin'] = np.sin(2 * np.pi * ts.dayofweek / 7)
    input_row['dayofweek_cos'] = np.cos(2 * np.pi * ts.dayofweek / 7)

    # Drop timestamp
    input_row.pop('timestamp')

    # One-hot encoding
    df_input = pd.DataFrame([input_row])
    df_encoded = pd.get_dummies(df_input)

    # Align columns with training features
    for col in X.columns:
        if col not in df_encoded:
            df_encoded[col] = 0
    df_encoded = df_encoded[X.columns]  # Ensure correct order

    return df_encoded


def predict_with_time_gps(input_row):
    """
    Full pipeline: preprocess + predict
    """
    df_prepared = preprocess_input_with_time_gps(input_row)
    prediction = model.predict(df_prepared)[0]
    return round(prediction, 2)




In [54]:
## Test

sample_input = {
    'timestamp': '2025-05-18 14:30:00',
    'event': 'venting',
    'temperature_C': 46.5,
    'CO2_ppm': 1900,
    'moisture_percent': 100.0,
    'mass_loss_percent': 12.3,
    'pH': 7.0,
    'forecast_temp_C': 33.5,
    'forecast_humidity_percent': 500.0,
    'forecast_condition': 'hazy',
    'humidity_sensor_percent': 40.0,
    'gps_latitude': 24.7136,
    'gps_longitude': 46.6753
}

pred = predict_with_time_gps(sample_input)
print("Predicted Remaining Hours:", pred)


Predicted Remaining Hours: 108.97


In [ ]:
!cp remaining_hours_predictor.pkl